In [2]:
from database.team_results import TeamResultsManager
import pandas as pd

In [3]:
file_path = 'data/jornadas/2024/seasson6/jornada7.xlsx'

Este bloque actualiza la tabla de resultados de equipos

In [5]:
# Agregando el resultado
df_resultado = pd.read_excel(file_path, skiprows=18, sheet_name='Partido', usecols="B:J", nrows=4)
df_resultado.dropna(how='all', inplace=True, axis=1)

trm = TeamResultsManager()

year = 2024
season = 6
week = 7


# check if the week result already exists
update_week = True
if trm.get_week_result(year, season, week):
    update = input("The week result already exists. Do you want to update it? (y/n): ")
    if update.lower() == 'y':
        trm.delete_week_result(year, season, week)
    else:
        update_week = False

if update_week:
    for row in df_resultado.iterrows():
        trm.add_team_result(
            year=year,
            season=season,
            match_week=week,
            team_name=row[1]["Equipo"],
            goals=row[1]["A favor"],
            goals_against=row[1]["En contra"],
            points=row[1]["Puntos"]
    )


Este bloque actualiza la de estadísticas agregando los stats de la jornada desde el excel de jornada.
- name
- team
- goals
- assists

Se debe leer luego desde la app para recoger 
- media
- mvp
- votes
- total_votes 

In [11]:
# Agregando las estadisticas de los jugadores
# Primero leemos el registro de jugadores
df_players = pd.read_excel(file_path, skiprows=1, nrows=28)
df_players = df_players.iloc[:, 1:].copy()
df_players.columns = ['Nombre', 'Equipo']
df_players.dropna(inplace=True, subset='Nombre', axis=0)

df_statistics = pd.read_excel(file_path, sheet_name='Partido', skiprows=3, usecols="L:O")
df_statistics.dropna(inplace=True, axis=0, how='all')
df_statistics.fillna(0, inplace=True)
df_statistics.drop(columns='Equipo', inplace=True)
df_statistics.columns = df_statistics.columns.str.strip()

df_excel = pd.merge(df_players, df_statistics, on='Nombre', how='outer')
df_excel.fillna(0, inplace=True)
df_excel.columns = ['name', 'team', 'goals', 'assists']


from database.players import PlayerManager
pm = PlayerManager()
fantasy_id_map = pm.get_fantasy_id()

df_excel['id'] = df_excel['name'].map(fantasy_id_map)
df_excel

c:\Users\Eddy.Tovar\Documents\Datos\personal\proyectos\fantasy_fulbo\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,name,team,goals,assists,id
0,Abu,Azul,2.0,1.0,1
1,Andres Diaz,Azul,0.0,0.0,4
2,Andres Freitas,Amarillo,0.0,1.0,5
3,Andres Ruiz,Azul,0.0,0.0,7
4,Arturo,Rojo,2.0,0.0,10
5,Bravo,Amarillo,0.0,0.0,14
6,Carlos,Negro,0.0,1.0,15
7,Chino,Rojo,1.0,0.0,16
8,De Tovar,Amarillo,0.0,0.0,19
9,Della Invitado,Rojo,0.0,1.0,65


Falta por agregar el código que escrapea desde la app. Lo he hecho en otro notebook para poder recuperar el histórico.